# Exploring tensor decompositions in pytorch
Simon Aertssen
12/03/2020

In [14]:
import torch
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


This notebook is an exploration of the different functions and methods necessary to perform the CANDECOMP/PARAFAC and Tensor-Train Decompositions. These will be gathered in a ```polution``` package later. The ```tensorly```package is a great resource: https://github.com/tensorly/tensorly/tree/master/tensorly

## 1. Useful products:

The $\textbf{Matrix Kronecker product}$ is defined as

\begin{align*} \boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \otimes \boldsymbol{Q}^{\:\mathrm{K} \times \mathrm{L}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{JL}}, \text { such that } r_{k+K(i-1), \: l+L(j-1)}=p_{i j} q_{k l} \end{align*}

Naive: use loops. Smart: reshape P and Q so that their dimensions do not overlap, then the elementwise or Hadamard product gives the desired result.

See [Wikipedia](https://en.wikipedia.org/wiki/Kronecker_product) for the examples.


In [487]:
def KronProd2D(P, Q):
    # Register dimensions.
    I, J = P.shape
    K, L = Q.shape
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    P = P.view(I, 1, J, 1)
    Q = Q.view(1, K, 1, L)
    
    R = P * Q
    return R.view(I*K, J*L)

def KronProd(P, Q):
    # This should work for higher order tensors.
    # Register and check dimensions.
    pshape = P.shape
    qshape = Q.shape
    if P.dim() != Q.dim():
        raise ValueError('Matrices should be of the same order: P.dim() =' + str(P.dim()) + ' != Q.dim() = ' + str(Q.dim()))
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    # Start with a list of ones and set dimensions as every even or uneven index.
    pindices = [1]*2*len(pshape)    
    pindices[::2] = pshape
    qindices = [1]*2*len(qshape)
    qindices[1::2] = qshape
        
    P = P.view(pindices)
    Q = Q.view(qindices)
    
    R = P * Q
    rshape = [p*q for p, q in zip(pshape,qshape)]
    return R.view(rshape)

Example:

In [488]:
P = torch.tensor([[1, -4, 7], [-2, 3, 3]])
Q = torch.tensor([[8, -9, -6, 5], [1, -3, -4, 7], [2, 8, -8, -3], [1, 2, -5, -1]])
R = KronProd(P, Q)
assert torch.all(R.eq(KronProd2D(P, Q)))

The $\textbf{Kahtri-Rhao product}$ is defined as

\begin{align*}
\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}}|\otimes| \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \odot \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{J}}, \text { such that } r_{k+K(i-1), j}=p_{i j} q_{k j}
\end{align*}

This can be seen as a column-wise kronecker product.

In [489]:
def KathRaoProd(P, Q):
    # Register and check dimensions
    pshape = P.shape
    qshape = Q.shape
    J = pshape[-1]
    if J != qshape[-1]:
        raise ValueError('Matrices should have the same number of columns: ' + str(J) + ' != ' + str(qshape[-1]))
    
    # Make R an empty tensor
    rshape = [p*q for p, q in zip(pshape,qshape)]
    rshape[-1] = J
    R = torch.zeros(rshape)
    for j in range(J):
        R[:,j] = KronProd(P[:,j], Q[:,j])
                
    # Tried with slicing but did not seem to work:
    # column_indices = list(range(J)) or slice(0,J)
    # R[:, column_indices] = KronProd(P[:, column_indices], Q[:, column_indices])
    return R


Example:

In [638]:
P = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
Q = torch.tensor([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
R = KathRaoProd(P, Q)
#print(R)

## 2. Mode-m Matricization

For some of the decompositions we will need tensor unfolding. The mode-$n$ matricization of a tensor $\mathcal{X} \in \mathrm{R}^{I_{1} \times I_{2} \times \ldots \times I_{N}}$ is denoted $\boldsymbol{X}_{(n)}$ and arranges the mode-$n$ fibres to be the columns of the resulting matrix.

\begin{align*}
\mathcal{X}^{I_{1} \times I_{2} \times \ldots \times I_{N}} \hspace{5mm}
\overset{\underset{\text{unfolding}}{\longrightarrow}}{\underset{\text{folding}}{\longleftarrow}} \hspace{5mm}
\boldsymbol{X}_{(n)}^{I_{n} \times I_{1} \cdot I_{2} \cdots I_{n-1} \cdot I_{n+1} \cdots I_{N}} 
\end{align*}

While writing the function underneath (```def Unfold```) it became clear that Kolda's definition was lacking some of the intuition behind torch tensors and their indexing. A good discussion can be found here: https://jeankossaifi.com/blog/unfolding.html

In [571]:
def Unfold(tensor, mode):
    # Notation sets minimal mode to be 1, but we want to map that to zero. We also cannot mode_n unfold a matrix with n-1 modes.
    if mode > len(tensor.shape) or mode < 0:
        raise ValueError('Tensor has order ' + str(len(tensor.shape)) + ', cannot give mode ' + str(mode))
    
    # Register tensor shape and mode size
    order = tensor.dim()
    tshape = tensor.shape    
    I_n = tshape[mode]
    
    # To get the prescribed matrix in Kolda et al, we need to perform a permutation of the axes.
    # The selected mode needs to be viewed as the first, and all other dimensions should reverse their order.
    # It took a lot of experiments to get this right.
    axes = list(range(order))
    axes.pop(mode)
    axes = axes[::-1]
    axes.insert(0, mode)
    
    return tensor.permute(*axes).reshape(I_n, -1)


In [572]:
# Following the example on p460:
# This definition of X gives wrong dimensions:
#X = torch.tensor([[ [1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12] ], [ [13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24] ]])

X = torch.tensor([[ [1,13], [4,16], [7,19], [10,22] ], [ [2,14], [5,17], [8,20], [11,23] ], [ [3,15], [6,18], [9,21], [12,24] ]])
print("X.shape = " + str(X.shape))
X1 = X[...,0]
X2 = X[...,1]
print("X_1 = \n" + str(X1.numpy()))
print("X_2 = \n" + str(X2.numpy()))

X.shape = torch.Size([3, 4, 2])
X_1 = 
[[ 1  4  7 10]
 [ 2  5  8 11]
 [ 3  6  9 12]]
X_2 = 
[[13 16 19 22]
 [14 17 20 23]
 [15 18 21 24]]


In [732]:
print(Unfold(X, 0).numpy())
print(Unfold(X, 1).numpy())
print(Unfold(X, 2).numpy())
# Note that the .numpy() is only here to represent the unfolded tensors in a nice format.

[[ 1  4  7 10 13 16 19 22]
 [ 2  5  8 11 14 17 20 23]
 [ 3  6  9 12 15 18 21 24]]
[[ 1  2  3 13 14 15]
 [ 4  5  6 16 17 18]
 [ 7  8  9 19 20 21]
 [10 11 12 22 23 24]]
[[ 1  2  3  4  5  6  7  8  9 10 11 12]
 [13 14 15 16 17 18 19 20 21 22 23 24]]


Do we need a ```Fold``` function? Update: yes. Perform reverse operations of Unfold.

In [847]:
def Fold(unfolded_tensor, mode, shape):
    if mode > len(shape) or mode < 0:
        raise ValueError('Folded tensor has order ' + str(len(shape)) + ', cannot give mode ' + str(mode))
    
    shape = list(shape)
    axis = shape.pop(mode)
    shape = shape[::-1]
    shape.insert(0, axis)
    
    return unfolded_tensor.reshape(shape).permute(*axes).reshape(shape)


In [848]:
print(Fold(Unfold(X, 0), 0, X.shape).numpy())
print(Fold(Unfold(X, 1), 1, X.shape).numpy())
print(Fold(Unfold(X, 2), 2, X.shape).numpy())

[[[ 1  4  7 10]
  [ 2  5  8 11]]

 [[ 3  6  9 12]
  [13 16 19 22]]

 [[14 17 20 23]
  [15 18 21 24]]]
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]

 [[19 20 21]
  [22 23 24]]]
[[[ 1  2  3]
  [13 14 15]
  [ 4  5  6]
  [16 17 18]]

 [[ 7  8  9]
  [19 20 21]
  [10 11 12]
  [22 23 24]]]


In [844]:
# Let's be absolutely shure:
print(X.shape)
print(X)
folded = Fold(Unfold(X, 0), 0, X.shape)
print(folded.shape)
print(folded)

#assert torch.all(X.eq(Fold(Unfold(X, 0), 0, X.shape)))

torch.Size([3, 4, 2])
tensor([[[ 1, 13],
         [ 4, 16],
         [ 7, 19],
         [10, 22]],

        [[ 2, 14],
         [ 5, 17],
         [ 8, 20],
         [11, 23]],

        [[ 3, 15],
         [ 6, 18],
         [ 9, 21],
         [12, 24]]])
torch.Size([3, 2, 4])
tensor([[[ 1,  4,  7, 10],
         [ 2,  5,  8, 11]],

        [[ 3,  6,  9, 12],
         [13, 16, 19, 22]],

        [[14, 17, 20, 23],
         [15, 18, 21, 24]]])


## 3. CP decomposition


Use the alternating least squares approach. Start with random matrices.

In [739]:
def CPD(tensor, rank=5, maxiter=25):
    # Note that we will perform n_iter_max iterations per tensor order!
    # Register input variables:
    tshape = tensor.shape
    order = tensor.dim()
    
    # Initialization: create a list of matrices to represent the input tensor: 
    # tensor = [t_1, t_2, .. , t_order]
    # Incorrect: ABC = [torch.rand((rank, rank))]*order
    ABC = []
    for o in range(order):
        ABC.append(torch.rand((tshape[o], rank)))
    ABC_indices = list(range(order))
    
    # Main loop: for every iteration, adjust every factor matrix
    for iteration in range(maxiter):
        for i, fmatrix in enumerate(ABC):
            # Get all the other fmatrices, temporarily remove the element:
            ABC.pop(i)

            # Construct V: use the first element ABC[0] and the loop over everything after with ABC[1::]
            V = ABC[0].t() @ ABC[0]
            for ABC_notfmatrix in ABC[1::]:
                V = V * (ABC_notfmatrix.t() @ ABC_notfmatrix)
            
            # Construct W (kr product of all matrices other way around):
            # take the last element of ABC and then loop over ABC[::-1] excluding the last element.
            # Example: ABC = [1,2,3,4] and ABC[::-1][1::] = [3,2,1] and ABC[1::-1] = [2,1]
            W = ABC[-1]
            for ABC_notfmatrix in ABC[::-1][1::]:
                #print(ABC_notfmatrix)
                #print(W.shape)
                #print(ABC_notfmatrix.shape)
                W = KathRaoProd(W, ABC_notfmatrix)
            #print(Unfold(tensor, i).shape)
            #print(W.shape)
            #print(torch.pinverse(V).shape)
            #print(i)
            fmatrix = Unfold(tensor, i) @ W @ torch.pinverse(V)
            
            # Push the lost fmatrix back in:
            ABC.insert(i, fmatrix)
    return ABC

In [740]:
# test:
tensor = torch.rand((10,15,20))
factor_matrices = CPD(tensor)
print(len(factor_matrices))

3


In [777]:
def Compose(factors):
    # Register dimensions of the tensor:
    order = len(factors)
    tshape = [x.shape[0] for x in factors]
    rank = factors[0].shape[-1]
    tensor = torch.zeros(tshape)
    
    #tmp_vecs = [x[:,1] for x in factors]
    
    for r in range(rank):
        tmp_vecs = [x[:,r] for x in factors]
        outer_prod = tmp_vecs[0]
        print(outer_prod.shape)
        for vector in tmp_vecs[1::]:
            print("vector = ", vector.shape)
            outer_prod = torch.ger(outer_prod, vector)
            print(outer_prod.shape)
        #tmp = torch.ger(factors[r][:,1].unsqueeze(2), factors[0][:,1].unsqueeze())
    #    tensor += 
    
Compose(factor_matrices)
    

torch.Size([10])
vector =  torch.Size([15])
torch.Size([10, 15])
vector =  torch.Size([20])


RuntimeError: vector and vector expected, got 2D, 1D tensors at /Users/distiller/project/conda/conda-bld/pytorch_1565272526878/work/aten/src/TH/generic/THTensorMath.cpp:886

In [754]:
y = torch.ones((5,3))
x = torch.zeros((5,8))
bmm = torch.bmm(x.unsqueeze(2), y.unsqueeze(1))
print(bmm.shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]])
torch.Size([5, 8, 3])
